# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Import what you need

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
import pandas as pd

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [3]:
driver = webdriver.Chrome()

In [4]:
driver.get("https://jportal.mdcourts.gov/license/index_disclaimer.jsp")

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages, or you can close and re-open Chrome.

- Check the checkbox, then submit the form to accept their terms of service

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

- *TIP: if something doesn't have anything special about it, xpath might be your best bet*

In [5]:
button = driver.find_element_by_id('checkbox')
button.click()

In [6]:
enter_button = driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]')
enter_button.click()

Now click the **Search License Records** link up top in the navigation to get to the search page.

- *TIP: Honestly you could also just visit the URL directly now since you filled out that terms of service thing*

In [7]:
search_button = driver.find_element_by_xpath('/html/body/table[1]/tbody/tr[2]/td[2]/table/tbody/tr/td[3]/a/img')
search_button.click()

### Perform your search

Pick "Statewide" for the jurisdiction dropdown, and `VAP%` into the Trade Name field. The `%` is a wildcard.

In [8]:
select = Select(driver.find_element_by_id('slcJurisdiction'))
select.select_by_visible_text('Statewide')

In [9]:
text_input = driver.find_element_by_id('txtTradeName')
text_input.send_keys('VAP%')


And now, of course, **submit the form**.

- *TIP: Since scrolling to buttons can be a pain, sometimes it's easier to select the form and use `.submit()` instead of `.click()`ing the button*

In [10]:
driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]').submit()

## (Try to) scrape the results

Let's start by just **printing this stuff**. We'll save it as a dataframe later on.

For now, just scrape **each store's name**, then cry a little. Fact: this is an impossible and miserable page. 

In [11]:
names = driver.find_elements_by_class_name('searchlistitem')


for name in names:
    print(name.text)

VAPE IT STORE II
VAPE IT STORE I
VAPEPAD THE
VAPE FROG
VAPE FROG


To avoid struggling with the search results page, we're going to use the **detail page** instead. Try to figure out how to select it and click it inside of your `for` loop.

- *TIP: Instead of just looking for an `a` or an `img`, you might want to look for one of its parents first, then click. This might affect the way you print out the shop's name, too*
- *TIP: Not all of them have links! You can wrap in try/except to skip it, or you can check to see if the shop's status is Pending.*

In [12]:
names = driver.find_elements_by_class_name('searchfieldtitle')

for name in names:
    link = name.find_element_by_tag_name('a')
    link.click()
    print(link)

<selenium.webdriver.remote.webelement.WebElement (session="15d28a85357dd7a1b7f7f064749e79aa", element="0.716466484975468-11")>


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=67.0.3396.79)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.16299 x86_64)


In [14]:
driver.back()

Okay, now let's get to action. For each result, **click the link to the detail page** and print out the following information:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

If it doesn't have a detail page, just print out the name and that's all we need.

- *TIP: When you're done getting the information, you probably want to click back to the search results*
- *TIP: You might enjoy `find_element_by_partial_link_text` to do that*
- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*

> **IMPORTANT NOTE:** This is doomed. It's useful to do, but your current process is doomed. Once you get a `stale element reference` error move on to the next cell.

### Stale message reference

Once you navigate away from a page, and you go back to it, you can't use the variables from the first time you were on the page. So, we got a list of results when we first visited, clicked to the details page, clicked back, and now our original list is "stale."

This is sad.

Let's try this again: loop through the results and create a dataframe with `name` and `url` columns. And yes, some of them won't have URLs.

In [48]:
results = driver.find_elements_by_class_name('searchfieldtitle')

vapeshops = []

for result in results: 
    
    vape = {}
    
    name = result.find_element_by_class_name('copybold')
    print(name.text) 
    vape['name'] = name.text
    
    try:
        url = result.find_element_by_tag_name('a').get_attribute('href')
        print(url)
        vape['url'] = url

    except:
        pass
    
    
    vapeshops.append(vape)
    vapeshops

VAPE IT STORE II
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=gsoqBtSzo%2Bk%3D
VAPE IT STORE I
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=kNwX1FHQ1GM%3D
VAPEPAD THE
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=RosfQ8b64%2BU%3D
VAPE FROG
VAPE FROG


In [49]:
df = pd.DataFrame(vapeshops)
df

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,VAPE FROG,NaN
4,VAPE FROG,NaN


### Getting all of the results

After you've looped through the results on one page, we're going to want to go to the next page! Add a line to make it click the 'Next' button down at the bottom

- *Tip: `find_element_by_partial_link_text` will be your friend*
- *Tip: You might need to do the scrolling thing to get it onto the screen (and by that I mean, you WILL need to, so you should)*

Confirm that it moves to the next page (it doesn't need to scrape anything yet)

In [59]:
results = driver.find_elements_by_class_name('searchfieldtitle')

vapeshops = []

for result in results: 
    
    vape = {}
    
    name = result.find_element_by_class_name('copybold')
    print(name.text) 
    vape['name'] = name.text
    
    try:
        url = result.find_element_by_tag_name('a').get_attribute('href')
        print(url)
        vape['url'] = url

    except:
        pass
    
next_button = driver.find_element_by_partial_link_text('Next')
driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
next_button.click()
    
    #vapeshops.append(vape)
    #vapeshops
    
    

VAPE LOFT THE
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=B4YT8nxDrZA%3D
VAPE N CIGAR
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wFWzu2JXLJM%3D
VAPE DOJO
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=oR1Io1sebEw%3D
VAPE HAVEN
VAPORS LOUNGE


### Wrapping with `while`

> Go back to the first page of results before you try to run this

You have a bunch of scraping code. It clicks the next button, then it stops. But you'd like it to go back up to the top! You can make that happen with a special `while` loop.

```python
while True:
    # Scrape your stuff
    # Click next button
```

This will go on FOREVER AND EVER until there is an error (when it can't find the Next button on the last page of results, you'll get an error).

- *Tip: Print out "Scraping a new page" every time you visit a new page, just to check that it's working*

In [64]:
goback_button = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr/td[1]/a/nobr')
driver.execute_script("arguments[0].scrollIntoView(true)", goback_button)
goback_button.click()

In [74]:
vapeshops = []

while True: 
    
    results = driver.find_elements_by_class_name('searchfieldtitle')

    for result in results: 
        vape = {}

        name = result.find_element_by_class_name('copybold')
        print(name.text) 
        vape['name'] = name.text

        try:
            url = result.find_element_by_tag_name('a').get_attribute('href')
            print(url)
            vape['url'] = url

        except:
            pass

        vapeshops.append(vape)
       
    next_button = driver.find_element_by_partial_link_text('Next')
    driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
    next_button.click()

    #print(vapeshops)
    print("Scraping new page")

VAPE IT STORE II
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=gsoqBtSzo%2Bk%3D
VAPE IT STORE I
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=kNwX1FHQ1GM%3D
VAPEPAD THE
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=RosfQ8b64%2BU%3D
VAPE FROG
VAPE FROG
Scraping new page
VAPE LOFT THE
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=B4YT8nxDrZA%3D
VAPE N CIGAR
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wFWzu2JXLJM%3D
VAPE DOJO
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=oR1Io1sebEw%3D
VAPE HAVEN
VAPORS LOUNGE
Scraping new page
VAPORFI SOUTHSIDE
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=%2BU3MygXFkLU%3D
VAPE DO JO
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ZlqlxTVVbWI%3D
VAPERTETMD
VAPERTEKMD
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=YUk7P1E2roY%3D
VAPORIOT
Scraping new page
VAPE PLUS
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?ow

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Next"}
  (Session info: chrome=67.0.3396.79)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.16299 x86_64)


### Making it perfect

> Go back to the first page of results before you try to run this

Wrap all of your code in a `try`/`except` so that it doesn't finish with an error and you'll be good to go.

**Confirm your list has all of the vape shops in it.** If not, check where you are creating your empty list (`[]`) - if you do it in the wrong spot, it will overwrite your list every time you visit a page.

In [78]:
try:
    
    vapeshops = []

    while True: 

        results = driver.find_elements_by_class_name('searchfieldtitle')

        for result in results: 
            vape = {}

            name = result.find_element_by_class_name('copybold')
            print(name.text) 
            vape['name'] = name.text

            try:
                url = result.find_element_by_tag_name('a').get_attribute('href')
                print(url)
                vape['url'] = url

            except:
                pass

            vapeshops.append(vape)

        next_button = driver.find_element_by_partial_link_text('Next')
        driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
        next_button.click()

        #print(vapeshops)
        print("Scraping new page")
    
except: 
    print("This is the last page.")

VAPE IT STORE II
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=gsoqBtSzo%2Bk%3D
VAPE IT STORE I
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=kNwX1FHQ1GM%3D
VAPEPAD THE
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=RosfQ8b64%2BU%3D
VAPE FROG
VAPE FROG
Scraping new page
VAPE LOFT THE
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=B4YT8nxDrZA%3D
VAPE N CIGAR
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wFWzu2JXLJM%3D
VAPE DOJO
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=oR1Io1sebEw%3D
VAPE HAVEN
VAPORS LOUNGE
Scraping new page
VAPORFI SOUTHSIDE
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=%2BU3MygXFkLU%3D
VAPE DO JO
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ZlqlxTVVbWI%3D
VAPERTETMD
VAPERTEKMD
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=YUk7P1E2roY%3D
VAPORIOT
Scraping new page
VAPE PLUS
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?ow

### Save this data as a csv

The filename should be `vape-shops-basic.csv`.

In [80]:
import pandas as pd

In [90]:
df = pd.DataFrame(vapeshops)
df

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,VAPE FROG,NaN
4,VAPE FROG,NaN
5,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicense...
6,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicense...
7,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicense...
8,VAPE HAVEN,NaN
9,VAPORS LOUNGE,NaN


In [91]:
df.to_csv("vape-shops-basic.csv", index=False)

In [98]:
df = pd.read_csv("vape-shops-basic.csv")
pd.set_option('display.max_colwidth', -1)
df

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=gsoqBtSzo%2Bk%3D
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=kNwX1FHQ1GM%3D
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=RosfQ8b64%2BU%3D
3,VAPE FROG,NaN
4,VAPE FROG,NaN
5,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=B4YT8nxDrZA%3D
6,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wFWzu2JXLJM%3D
7,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=oR1Io1sebEw%3D
8,VAPE HAVEN,NaN
9,VAPORS LOUNGE,NaN


# Okay, let's scrape!

All right, get the actual data!

### Look at the URL of your first row

- *TIP: Remember `pd.set_option('display.max_colwidth', -1)` will let you see alllll of your strings*

In [160]:
pd.set_option('display.max_colwidth', -1)
df.head(1)

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=gsoqBtSzo%2Bk%3D


### Use Selenium to visit that page

In [99]:
driver.get("https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=gsoqBtSzo%2Bk%3D")

### Now, just like you did before, grab the additional data

You should probably save it into a dictionary! Don't try to put it into the dataframe yet, though. You want:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

.

- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*
- *TIP: If you've gotten addicted to xpath, total amount paid and issued by might not work with it when doing other shops. You'll want to test it!*

In [123]:
mailing_address = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text.replace("\n", " ")
location_address = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text.replace("\n", " ")
license_info = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text.split("\n")
amount_paid = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr[8]/td[2]').text
issued_by = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[3]/tbody/tr[2]/td').text.split("\n")

print(mailing_address)
print(location_address)
print(license_info)
print(amount_paid)
print(issued_by)

AMIN NARGIS 1104 PLANTERS PLACE SALISBURY, MD 21804
VAPE IT STORE II 1015 S SALISBURY BLVD SALISBURY, MD 21801
['License Status: Issued', 'License No.: 22375606', 'Control No.: 22884439', 'Date of Issue: 4/27/2018', 'Months Paid: 12', 'Exp. Date: 4/30/2019', 'Subdivision: 22 Salisbury']
$ 41.00
['WICOMICO COUNTY, CLERK OF CIRCUIT COURT', 'P.O. BOX 198', 'SALISBURY, MARYLAND 21803-0198']


### Move all of this into one cell

It should visit the URL, then grab the data and put it into a dictionary.

In [157]:
    driver.get("https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=gsoqBtSzo%2Bk%3D")

    
    mailing_address = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text.replace("\n", " ")
    location_address = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text.replace("\n", " ")
    license_info = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text.split("\n")
    amount_paid =driver.find_elements_by_class_name('rt')[-1].text
    issued_by = driver.find_elements_by_class_name('tablecelltext')[8].text.split("\n")

    pd.Series({
        'mailing_address': mailing_address,
        'location_address': location_address,
        'license_info': license_info,
        'amount_paid': amount_paid,
        'issued_by': issued_by
    })

mailing_address     AMIN NARGIS 1104 PLANTERS PLACE SALISBURY, MD 21804                                                                                                               
location_address    VAPE IT STORE II 1015 S SALISBURY BLVD SALISBURY, MD 21801                                                                                                        
license_info        [License Status: Issued, License No.: 22375606, Control No.: 22884439, Date of Issue: 4/27/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 22 Salisbury]
amount_paid         $ 41.00                                                                                                                                                           
issued_by           [WICOMICO COUNTY, CLERK OF CIRCUIT COURT, P.O. BOX 198, SALISBURY, MARYLAND 21803-0198]                                                                           
dtype: object

### Change it into a function

You'll want to have this function accept a `row`, and send back a `pd.Series`. You can just use `pd.Series(your_dictionary)` (but it better have a better name than `your_dictionary`!).

- *TIP: Make sure you `return` something!*
- *TIP: Make sure you change everything to reflect the row's url, not the URL you typed in*

In [165]:
def get_vape_info(row):

    try: 
        driver.get(row['url'])
        
        mailing_address = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text.replace("\n", " ")
        location_address = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text.replace("\n", " ")
        license_info = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text.split("\n")
        amount_paid =driver.find_elements_by_class_name('rt')[-1].text
        issued_by = driver.find_elements_by_class_name('tablecelltext')[8].text.split("\n")

        return pd.Series({
            'mailing_address': mailing_address,
            'location_address': location_address,
            'license_info': license_info,
            'amount_paid': amount_paid,
            'issued_by': issued_by
        })
    
    except:
        return pd.Series({})

### Use your dataframe and `.apply` to pull all of the data from the vape shops

Once you know it's working, use the whole 

- *TIP: Try using it with `.head(3)` first*
- *TIP: You'll want to use `.apply` with your new function*
- *TIP: Issued By and Total Paid are goign to give you problems if you tried to use xpath! Try checking the classes and think about find_elementSSSSS and working backwards instead of forwards.*
- *TIP: You might need a `try`/`except`*
- *TIP: Make sure you're using `axis=1`*
- *TIP: Use `.join` the big thing with all of the `dfs` - make sure you name them right!*

In [166]:
new_df = df.apply(get_vape_info, axis=1)
new_df.head(3)

,amount_paid,issued_by,license_info,location_address,mailing_address
0,$ 41.00,"[WICOMICO COUNTY, CLERK OF CIRCUIT COURT, P.O. BOX 198, SALISBURY, MARYLAND 21803-0198]","[License Status: Issued, License No.: 22375606, Control No.: 22884439, Date of Issue: 4/27/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 22 Salisbury]","VAPE IT STORE II 1015 S SALISBURY BLVD SALISBURY, MD 21801","AMIN NARGIS 1104 PLANTERS PLACE SALISBURY, MD 21804"
1,$ 24.00,[ ],"[License Status: Issued, License No.: 22375605, Control No.: 22591855, Date of Issue: 4/27/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 22 Salisbury]","VAPE IT STORE I 1724 N SALISBURY BLVD UNIT 2 SALISBURY, MD 21801","AMIN NARGIS 1104 PLANTERS PLACE SALISBURY, MD 21804"
2,$ 94.00,[ ],"[License Status: Issued, License No.: 02304705, Control No.: 02685930, Date of Issue: 4/05/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 02 Anne Arundel County]","VAPEPAD THE 2299 JOHNS HOPKINS ROAD GAMBRILLS, MD 21054","ANJ DISTRIBUTIONS LLC 2504 ORCHARD KNOLL WAY ODENTON, MD 21113"


## Save as `vape-total.csv`

Make sure you don't save the index! Open it up in a text editor or Excel to make sure it's correct.

In [167]:
new_df.to_csv("vape-total.csv", index=False)